# PID Controllers

In this notebook, I will explain how each of proportional (P), derivative (D), and integral (I) controls were used in this research. First, I will provide an overview for the objective of each control. Next I will explain our particular implementation for each control using 1 and 2 sensors. I am including an analysis of benefits and liabilities for each. The Python programs that are in the master directory already have these controls implemented, therefore most of this notebook gives the conceptual background and research approach used to implement them.

*Note: This notebook assumes that you have already defined "steer" for the EV3. Do not proceed without having already understood steering the EV3.*

Before moving forward, it is important to define that a "**target**" intensity refers to your desired intensity value on the printed gradient. Irrespective of controller, every time you read in a value from the color sensor, you should compare it to a set target value and make a steering decision based on that. A reasonable target value is obtained by measuring the extremes of the color sensor intensities and averaging them. Low values correspond to dark areas on the printed gradient, while high values correspond to light areas on the printed gradient.

From my tests, the printed gradient most frequently had a lowest color intensity of 4, and a highest color intensity of 84. Therefore a reasonable target would be (84-4)/2=40. In practice, you may observe that slightly lower targets will actually perform better depending on controller, but for now, measure the reasonable target as listed above and use that.

## Proportional Control
The **proportional control**, or simply the "**P control**," is used to regulate the steering of the EV3 based on instantaneous feedback from the color sensor. The values that are received from the color sensor should be compared with a target value, and based on that result the EV3 should steer. The robot is using the current color sensor value in order to correct its position on the gradient to achieve a specific target value. Observe the following pseudocode for the process below:

### Flow of P Control

---
                        while running:
                            0. Get intensity value from color sensor
                            1. Compare intensity value with target value
                            2. Adjust steering for EV3
                            
---


Now, I know what you're thinking. "*How exactly do I adjust the steering for the EV3?*" There are three components to this question, and each will be addressed below. This is a nontrivial task and is crucial to any implementation of a PID controller.

So, the first part of this question is purely intuitive. You should steer to the direction that brings you closer to the target intensity. Since the printed gradient is a line, you are steering either left or right depending on the current reported intensity of the color sensor. This is represented through simple if-elif-else decision making in your Python code.

The second part of this question is not so intuitive. You are trying to translate the unitless quantity reported by the EV3 color sensor into a measure for adjusting the steering. If you are using one color sensor on the robot, then it makes sense to subtract the target intensity from the reported intensity in order obtain the difference. The magnitude of the difference will tell you how much to steer. It is best to multiply this difference by a constant parameter in order to achieve the best control. The sign of the difference will tell you which direction to steer in.

The third part of this question is how often you should update the steering. It is best to surround the above code in a while-loop in order to guarantee that you are updating as frequently as possible.

Now, there may still be some confusion as to how to choose a parameter. A systematic approach to this problem is to first make the paramter 1, then run the robot and see how it steers. If there is oscillatory behavior, it is probably oversteering and the constant can be reduced. On the other hand if there is a noticeable lack of steering, then the constant needs to be increased.

This procedure will guarantee an adequate implementation of the P control.

Now, if you wish to use two color sensors on the EV3, then there needs to be a slight modification to the method for translation of sensor intensities into a measure for adjusting the steering. Instead of comparing to a target value, subtract the sensor intensities from each other. This difference in intensity values functions similarly to the difference in sensor and target above, and the programming of this solution follows the above method.

## Derivative Control
The **derivative control**, or "**D control**," takes into consideration the previous values of the color sensor. Previous sensor intensity values and their timestamps are stored in a list. These sensor intensity and timestamp pairs can then be compared with the current readings from the color sensor in order to gauge a measure of how fast the intensity is changing over time. This provides a new measure for us to adjust the steering, and is known as the D control. It is important to note that the D control is not meant to be used independently of the P control, but is rather taken in a linear combination with the P control. This combination is usually referred to as a "**PD control**." Let us observe the PD control below:

### Flow of PD Control:
---
                        while running:
                            0. Get intensity value from color sensor and record timestamp
                            1. Store intensity value and timestamp pair in a list
                            2. P control: compare intensity value with target value
                            3. D control: compare intensity value with previous intensity values
                            4. Use a linear combination of P and D controls to adjust steering

---
Now, you may be wondering again, "*How do I adjust steering for the EV3?*" The P control is unmodified from the above section of this notebook. The D control provides a measure of the change in intensity value over a time period. The most intuitive implementation of this is to subtract the immediate previous color sensor value from the curent sensor value and divide over the timestep between the two, as follows: 

---
                            D = (curent_intensity - previous_intensity)/timestep
                            
---
                            
Even though this may seem intuitive, in actual implementation, it has liabilities due to the nature of the discrete sensor values and small timestep. Dividing by a small timestep induces magnification of minor pertubations in the data, and therefore is unreliable to be used in general. In addition, the small timestep is not constant since linux is not a hard realtime system.

So how exactly should you calculate the D control? Store a list of pairs of intensity values and their timestamps. Perform the above calculation, but instead of using the immediately previous value, use a value that is 8 values in the past. It would be good to define a parameter for how many values in the past you are using, and test different numbers for that parameter, in order to draw a systematic comparison. The code would look as follows:

---
                            previous_values = []
                            #Add sensor intensities to list
                            #Use count modulus 8 function to update elements in list
                            D = (curent_intensity - 8th_previous_intensity)/timestep
                            
---
If the timestep between each sensor reading is approximately 0.05 seconds, then 8 times that will be 0.40 seconds. This is a much better timestep to divide by, but still needs further investigation in real implementation.

## Integral Control

The **integral control**, or "**I control**," keeps a running average of intensity values in order to influence the measure by which you adjust the steering. Just as with the D control, the I control is not to be used independently, but is rather used in a linear combination with the P control. This combination is usually referred to as a "**PI control**." Let us observe the PI control below:

### Flow of PI Control:
---
                        while running:
                            0. Get intensity value from color sensor
                            1. P control: Compare intensity value with target value
                            2. I control: Update average with intensity value
                            3. I control: Compare average with target value
                            4. Use a linear combination of P and I controls to adjust steering

---
Now, you may be wondering, "*How do I update the average with the intensity value?*" The I control is an attempt to integrate the intensity values. It is robust because it resists drastic change during changes of intensity. One way to build such a control is as follows:

---

                            I = (1-μ)*I + μ*current_intensity
---
In this case, we need μ to be a small, positive, float. The most intuitive value for μ is less than or equal to the timestep of each color sensor reading, so 0.05 or less. You can see that now this control will retain 95 percent of the previous average and only integrate in 0.05 percent of the current intensity, so it will be hard to create a deviation under minor inaccuracies in the data.

Just as with the P control, you will need to compare the I control value with the target value in order to justify an adjustment of the steering.

## PID Control

The combination of each of the Proportional, Integral, and Derivative controls is referred to as a "**PID control**." The flow of the the PID control can be described by joining the above PI and PD controls. We have experimented with this control in the past, but it still needs further investigation. In order to establish a good PID control, you should first implement the P control and systematically test different constants for the steering adjustment. Next repeat this procedure after adding another control. In the past, it is hard to observe how the constant modifications are impacting the EV3, therefore it is useful to come up with scenarios that can be used to validate whether a control is getting better or worse. Dr. Seibold has written a Matlab motion simulator for this research, which can be used to see if experimental results on the EV3 agree with theoretical results on the simulator. Additionally, you can store the values from each iteration of the program in a csv file and later graph everything versus time in order to determine if there is a noticeable correlation between different constants.

When coding the above controls, it is advantageous to converge on a single code structure. The following structure is what I have been using when coding the robots. This visualization helps see the inputs and outputs at each level of the code.

![](ModularCode.png)

This picture aims to summarize how the PID controls are used to caclualte the steering adjustment. After the initial distance is sensed by the color sensor, then processing is done to obtain the derivative and integral controls, which  are then combined in a controller and output a steering adjustment for the actuation of the motors.

## Testing Controls

### Matlab Simulator
Dr. Seibold has written a Matlab motion simulator for this research, which can be used to see if experimental results on the EV3 agree with theoretical results on the simulator. This is useful for making predictions and test cases, especially if you are doing work outside of the lab. It could be useful to revisit this program and see if there are disagreements between the program and actual results.
### Tracking Controls
In order to preserve the modular code structure above, it is important to not remove a component of the code when testing new controllers and code. The D, I or any other test controls can be programmed into the Python script, but not actually used in steering adjustments. The way to do this is to set the constant for a particular control to 0 so that it will not impact the steering adjustment. Therefore, the computations for each control will be executed, but the control will not be used. It is useful to see how the data looks when a control is tracked versus when it is actually used. It is also important to not remove segments of the code between doing this, but rather keep the computations constant so that the differences in computation time do not impact performance.
### Graphing Data
The Python script currently saves all of the control values into  a csv file. This csv file can be exported and uploaded to various programs to visualize the behavior of the EV3. If you are doing many tests, it may be useful to remember a few files and plot them in Excel for efficiency. If you are trying to showcase an interesting result, I would recommend using Tableau, a visualization software that is easy-to-use and makes nice charts.

Below I have provided an example of a graph in Excel that features many of the controls that we have tested to date.

![](ExampleGraphTimeVsControls.png)

Observe that the intensity reading is usually in the range of 40 to 60. It has minor osciallations in its graph versus time, which makes sense because the robot is perpeutally adjusting its steering to reach a target. The integration of that intensity reading with respect to time is much smoother and resistant to the noise in the sensor readings. This is why it provides stability in steering. The D control was omitted from the graph because it is very sensitive to noisy data. We can see that the program is functioning as intended if these testing graphs match the expected results. 

The values labelled "W Control" and "Steering Angle" are other controls and data that is being tracked. I have referred to the "Steering Angle" as a Steering Adjustment throughout this notebook, as it more accurately describes that measurement since you are not actually steering by an angle. (Steering Angle is rather a misnomer.) The W control, or rather Steering Adjustment Control, aims to eliminate the noise in steering adjustments by integrating the steering adjustments with respect to time.

We can also visualize the the motion of the robot as it moves around the printed gradient. The graph above starts at the center of the straight edge on the oval printed gradient, then as it moves around the sharp turn, the steering angle increases in magnitude. As the robot returns to the straight side, it exhibits a smooth transition to nearly a 0 steering angle, but also has a noticeable oscillation as it moves between the different white squares in the lab.

## Steering Adjustment Control

This control was used in early testing in order to see if it were advantageous to control the steering adjustments that each iteration of the PID+ controls generate. As stated above, the **Steering Adjustment Control**, or aims to eliminate the noise in steering adjustments by integrating the steering adjustments with respect to time.
### Flow of Steering Adjustment Control:
---
                        while running:
                            0. Get steering adjustment
                            1. Update steering adjustment average with steering adjustment
                            2. Use linear combination of steering adjustment and steering 
                               adjustment average to adjust steering

---
This control seems to intuitively provide stability to the steering of the EV3, however it needs to be investigated further once the main PID controls are properly tuned.